# Generate Training Data

We wish to speed up the process of having a human being name entities that they recognize in a paragraph, and having their positions in the paragraph identified and placed in a syntax usable by spaCy's NER training routine.

## Open PDF file, extract page two and display as sentences

In [14]:
import spacy
import PyPDF2
nlp = spacy.load('en_core_web_sm')

#Open PDF file for reading
# pdfFile = open("BarCvDescLJ11.pdf", mode="rb")
pdfFile = open("Data/UIdaho2019/small-grains-report_2019.pdf", mode="rb")
pdfReader = PyPDF2.PdfFileReader(pdfFile)

# Select a page to work on
pageNumber = 49

# Get text
OnePage = pdfReader.getPage(pageNumber-1) #0-based count
OnePageText = OnePage.extractText()

# Close PDF file
pdfFile.close()

# Remove newlinesxmx82k-&a. It appears multiple newlines together makes
# Spacy think that is the end of a sentence. The PDF reader reads the text in
# an odd fashion
OnePageText = OnePageText.replace('\n','')

# create a spaCy doc object from the page and break it into sentences
doc = nlp(OnePageText)
l=0
for sent in doc.sents:
     print(l, ": ", sent)
     l = l+1


0 :   other soft white spring wheat varieties over four irrigated locations, but in 2019 (Table 42) irrigated yields were below average and similar to Louise.
1 :  Under dryland conditions, yield was similar to SY Saltese and WB6430
2 :  (Table 47).
3 :  Ryan has Hessian fly resistance, tolerance to low acid / high aluminum soils, and HTAP (high temperature adult plant) resistance to stripe rust.
4 :  Ryan was early to heading, similar to UI Pettit, was shorter than average, had good test weight and resistance to lodging.  
5 :  Seahawk (WA8162)  a soft white spring wheat released from Washington State  ram in 2014 adapted to dryland and irrigated production areas.
6 :  Seahawk has resistance to Hessian fly, is very resistant to stripe rust, and one of the least susceptible soft white spring wheats to FHB.
7 :  Seahawk has tolerance to high aluminum, low pH soils.
8 :  Yield and test weight has been one the highest of all currently available soft white springs, with similar to UI Stone

## Read in Per-line named entity file and match entities to sentence positions.

In [16]:
import re
import csv
import pandas as pd

#fname = "Data/DavisLJ11/barley_p"+str(pageNumber)+"_ner.txt"
fname = "Data/UIdaho2019/barley_p"+str(pageNumber-40)+"_ner.txt" # Note p1 = 41 in the document

# Covert the nlp senetence generator into a list of sentences
sentences = list(doc.sents)

# Open the file of manually matched pairs (sentence # <tab> word phrase <tab> named entity)
# e.g.:
#  0      AC Metcalfe     CVAR
#  0      two-rowed       TRAT
#  0      barley          CROP
#  1      Agri-Food Candada   ORG
#  1      1997    DATE
file = open(fname)
reader = csv.reader(file, delimiter='\t', quoting=csv.QUOTE_NONE)
data = list()

for row in reader:
    try:
        (sentIndex, phrase, label) = row
        sent = sentences[int(sentIndex)].string.rstrip()
        
        # find all instances of the 'phrase' in the 'sent'.
        iter = re.finditer(r"\b"+phrase+r"\b", sent)
        indices = [m.start(0) for m in iter]
        
        # check to make sure the phrase the user said was there was indeed found
        if len(indices) == 0:
            raise ValueError
                
        # print out all instances
        for i in indices:
#            print(sentIndex, sent, phrase, "("+str(i), i+len(phrase), "'"+label+"')")
            data.append([sentIndex, sent, phrase, "("+str(i)+", "+str(i+len(phrase))+", '"+label+"')"])
            
    except:
        print("Handle manually: ", row)
        
df = pd.DataFrame(data, columns = ["Index", "Sentence", "Phrase", "MatchInfo"])
print(df)


    Index                                           Sentence    Phrase  \
0       0   other soft white spring wheat varieties over ...      soft   
1       0   other soft white spring wheat varieties over ...     white   
2       0   other soft white spring wheat varieties over ...    spring   
3       0   other soft white spring wheat varieties over ...     wheat   
4       0   other soft white spring wheat varieties over ...      2019   
..    ...                                                ...       ...   
167    34  UI Stone was selected for good end-use quality...       FHB   
168    35  In 2019, UI Stone yielded similar to Seahawk a...      2019   
169    35  In 2019, UI Stone yielded similar to Seahawk a...  UI Stone   
170    35  In 2019, UI Stone yielded similar to Seahawk a...   yielded   
171    35  In 2019, UI Stone yielded similar to Seahawk a...   Seahawk   

            MatchInfo  
0     (7, 11, 'TRAT')  
1    (12, 17, 'TRAT')  
2    (18, 24, 'TRAT')  
3    (25, 30, '

## Create a function to clean up overlapping intervals

In [17]:
import re
coordRegex = re.compile(r'(\d+), (\d+)')

def sortByStart(coords):
    """For use in sort routines, return object with lowest (X,Y) values"""
    # split out coordinates that come in as (5, 7, 'CVAR')
    mo = coordRegex.search(coords)
    return(int(mo.group(1)))

def overlaps(coord1, coord2):
    """Check if coordinates of the form 5, 7, 'CVAR' and 32, 46, 'TRAT' overlap"""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
    
    if ((coord1High >= coord2Low) and (coord1Low <= coord2Low) or
        (coord2High >= coord1Low) and (coord2Low <= coord1Low)):
        return True
    else:
        return False

def keepFirst(coord1, coord2):
    """Given overlapping coordinates, return the wider encompassing one."""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
 
    if (int(coord1High) - int(coord1Low)) >= (int(coord2High) - int(coord2Low)):
        return True
    else:
        return False

# print("Should be false:", overlaps("(5, 7, 'CVAR')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(26, 46, 'TRAT')"))
# print("Keeper:", keepFirst("34, 46, 'TRAT'", "34, 46, 'TRAT'"))

def cleanIntervals(inputString=""):
    """order intervals like (5, 7, 'CVAR'), (32, 46, 'TRAT'), (26, 46, 'TRAT') and remove overlapping ones."""
    inputString = inputString.lstrip("(").rstrip(")")
    intervalList = inputString.split("), (")
    intervalList.sort(key = sortByStart)
#    print("Sorted Interval List:", intervalList)

    # Pairwise compare every interval in the list to every other interval to check overlap
    keeperList = [True]*len(intervalList) # Logic array to determine if each interval should be kept
    i=0
    for interval1 in intervalList:
        for interval2 in intervalList:
            if interval1 == interval2:
                if intervalList.index(interval1) != i: # when both are the same we reject the higher one
                    keeperList[i] = False
            else:
                if overlaps(interval1, interval2) and keepFirst(interval1, interval2) == False:
                    keeperList[i] = False
        i = i+1
        
#    print("keeperList:", keeperList)
   
    # Build up the return interval list
    returnStr = "("
    for interval, isKeeper in zip(intervalList, keeperList):
        if isKeeper:
            returnStr = returnStr + interval + "), ("
    return (returnStr.rstrip("), (") + ")")
        
# cleanIntervals("(5, 7, 'CVAR'), (32, 46, 'TRAT'), (5, 9, 'CVAR'), (48, 55, 'ORG'), (26, 46, 'TRAT')")
# cleanIntervals("(0, 8, 'CVAR'), (0, 5, 'CVAR'), (21, 26, 'PLAN'), (32, 37, 'CVAR')")
cleanIntervals("(0, 12, 'CVAR'), (39, 49, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (140, 150, 'CVAR'), (181, 187, 'CVAR'), (181, 187, 'CVAR')")


"(0, 12, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (181, 187, 'CVAR')"

## Aggregate all matches for each sentence on a single line and output in spaCy training format

In [18]:
# use Pandas dataframes to aggregate all entity matches together for a single sentence
agg_rules = {'Sentence': 'first', 'Phrase': 'first', 'MatchInfo': lambda x: ', '.join(x)}
res = df.groupby('Index').agg(agg_rules)
#print(res)

# Now format it just like what is needed for the spaCy training module: 
# E.g.:
# ('Eight-Twelve is a six-rowed winter feed barley', {'entities': [(0, 12, 'CVAR'), (18, 27, 'TRAT'), (28, 39, 'TRAT'),(40, 46, 'CROP')]}),
records = res.to_dict('records')
print("TRAIN_DATA = [")
maxr = len(records)
for i in range(0,maxr):
    print("    ('"+records[i]['Sentence']+"', {'entities': ["+cleanIntervals(records[i]['MatchInfo'])+"]})", end='')
    if (i == maxr-1):
        print()
    else:
        print(",")

print("]")

TRAIN_DATA = [
    (' other soft white spring wheat varieties over four irrigated locations, but in 2019 (Table 42) irrigated yields were below average and similar to Louise.', {'entities': [(7, 11, 'TRAT'), (12, 17, 'TRAT'), (18, 24, 'TRAT'), (25, 30, 'CROP'), (79, 83, 'DATE'), (95, 111, 'TRAT'), (146, 152, 'CVAR')]}),
    ('Under dryland conditions, yield was similar to SY Saltese and WB6430', {'entities': [(26, 31, 'TRAT'), (47, 57, 'CVAR'), (62, 68, 'CVAR')]}),
    ('Plant height is a little above average and heading date 2-4 days later than average.', {'entities': [(0, 12, 'TRAT'), (43, 55, 'TRAT')]}),
    ('Seahawk may have a tendency to lodge under high production practices.', {'entities': [(0, 7, 'CVAR'), (31, 36, 'TRAT')]}),
    ('SY Saltese (SY3024-2)  a soft white spring wheat released in 2016 by Syngenta Cereals.', {'entities': [(0, 10, 'CVAR'), (12, 20, 'ALAS'), (25, 29, 'TRAT'), (30, 35, 'TRAT'), (36, 42, 'TRAT'), (43, 48, 'CROP'), (61, 65, 'DATE'), (69, 85, 'ORG')]}),
  

In my current process, I am writing the above content to a file e.g., `Data/DavisLJ11/barley_p5_td.py`, adding any manual corrections (usually PED and JRNL entries) and then running the script `python3 py2json.py --doc 'BarCvDescLJ11.pdf' --url 'https://smallgrains.ucdavis.edu/cereal_files/BarCvDescLJ11.pdf' --chunk 5 Data/DavisLJ11/barley_p5_td.py Data/DavisLJ11/barley_p5_td.json` to create the JSON file for Training.